In [ ]:
cd ..

In [31]:
import pytesseract
from PIL import Image
import pandas as pd
from pdf2image import convert_from_path
import os

In [32]:
from PIL import Image, ImageFilter, ImageEnhance

def preprocess_image(image: Image):
    # Convert to grayscale
    image = image.convert('L')
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Apply sharpening
    image = image.filter(ImageFilter.SHARPEN)
    return image

In [33]:
TESSERACT_OCR_PATH = os.getenv('TESSERACT_OCR_PATH')
POPPLER_PATH = os.getenv('POPPLER_PATH')

In [42]:
images = convert_from_path("test_data\Amekor_Invoice_01.pdf", dpi=500, poppler_path=POPPLER_PATH)
# images = convert_from_path("test_data\Boyang_Invoice_01.pdf", poppler_path=POPPLER_PATH)

In [43]:
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500>]

In [44]:
all_data = []

In [45]:
for image in images:
    image_processed = preprocess_image(image)
    text = pytesseract.image_to_string(image)
    print(text)

\y

VIVICA A FOX _ TEL (610) 825-6747 FAX (610) 834-8427

HAIR COLLECTION

AMEKOR INDUSTRIES

500 BROOK ROAD CONSHOHOCKEN, PA 19428

Sold to:
[MNO9] HAIR DEPOT Me Il [N60][B]
[MNO9] JIN YOUNG L

590 UNIVERSITY AVE. WwW

ST PAUL MN 55103

UNITED STATES

INFO@AMEKOR.COM

INVOICE

Order No: 235880
Invoice No: SI-168211
Invoice Date: 14 Mar 2024

Payment Due: 13 May 2024

Ship to:

[MNO9] HAIR DEPOT PLUS | [N60][B]

[MNO9] JIN YOUNG L

590 UNIVERSITY AVE Ww
ST PAUL MN 55103
UNITED STATES

Salesrep. Customer No. Customer P/O Terms Ship Via
Andrew Cho MNO9 3/14/24 7 MILE N60 UPS Ground
(651-290-0362) Q

Qty Product Color Description UPC UPC Barcode Price Amount

2 AWBANDI 4 SSawitueweerpance,  e'et09186 TT ATTA TINUATMTTAM TT] $9.78 $1980

2 AWBANDI 1B Sa wirueweerganc 7 ee'19175 TT ATTEN TITV MIM TM 89-78 $1980

2 AweAND! FsiBi90 eeseiToereNaER NS TereroorrroHMTMMIIAINAUMIINIMIKE = $272 8195

# eusouo 8 pseesatetenc we "er TMT" 88
BAN

2 AWBOLD SAT Sesorew ousLwh eet %47824 TT ATTIATTATTAAV

In [41]:
image.show()

In [63]:
image_processed.show()

In [74]:
ImageEnhance.Contrast(image.convert('L')).enhance(2).filter(ImageFilter.SHARPEN).show()

In [10]:
import re

In [11]:
table_data = re.search(r"ITEM QTY\.ORD\|QTY\.SHIP DESCRIPTION UNIT PRICE AMOUNT(.*)Item Total", text, re.S)

In [12]:
table_data

<re.Match object; span=(398, 715), match='ITEM QTY.ORD|QTY.SHIP DESCRIPTION UNIT PRICE AMOU>

In [13]:
if table_data:
    table_rows = table_data.group(1).strip().split('\n')

    # Process each row
    for row in table_rows:
        # Here we assume the structure is consistent as shown in your example text
        # Adjust the regular expression as per the actual data format
        row_data = re.search(r"(\w+)\s+(\d+)\s+(\d+)\s+#(.+):\d+, #\d+\s+(\d+\.\d+)\s+(\d+\.\d+)", row.strip())
        if row_data:
            item = {
                'Item Name': row_data.group(1),
                'Quantity Ordered': int(row_data.group(2)),
                'Quantity Shipped': int(row_data.group(3)),
                'Description': row_data.group(4),
                'Unit Price': float(row_data.group(5)),
                'Amount': float(row_data.group(6))
            }
            print(item)

{'Item Name': 'DENISHA', 'Quantity Ordered': 2, 'Quantity Shipped': 2, 'Description': 'SP1B/30', 'Unit Price': 18.75, 'Amount': 37.5}
{'Item Name': 'JECKA', 'Quantity Ordered': 2, 'Quantity Shipped': 2, 'Description': 'SP1B/30', 'Unit Price': 18.95, 'Amount': 37.9}
{'Item Name': 'JINNY', 'Quantity Ordered': 2, 'Quantity Shipped': 2, 'Description': '1BN27', 'Unit Price': 20.5, 'Amount': 41.0}
{'Item Name': 'LASLEMY', 'Quantity Ordered': 2, 'Quantity Shipped': 2, 'Description': '4', 'Unit Price': 14.95, 'Amount': 29.9}
{'Item Name': 'SLBONE', 'Quantity Ordered': 4, 'Quantity Shipped': 4, 'Description': 'BT4007', 'Unit Price': 12.75, 'Amount': 51.0}
{'Item Name': 'wyo', 'Quantity Ordered': 1, 'Quantity Shipped': 1, 'Description': '773314', 'Unit Price': 15.95, 'Amount': 15.95}


In [24]:
table_data.group(1).strip().split('\n')[0]

'DENISHA 2 2 #SP1B/30:2, #1 18.75 37.50'

In [46]:
custom_config = r'--oem 3 --psm 6 --dpi 500'
text_1 = pytesseract.image_to_string(image, config=custom_config)

In [47]:
text_1

'Qty Product Color Description UPC UPC Barcode Price Amount\n2 GP REO Es azecs = TAMTMMTNMTMINTTN $275 5°\n6,10,12-1P\n1 Whrtney. 778089 oe etocseount — eet499176 0 TANTTAINTTATATITT MITTIN ITT S88.9° $89.09\nV COMBINATION W/ CURLY\nBANG\n2 yaya 1 sc evmpcwomtevece reverses MINIMUM $1222\nSpecial !\n$1650)\n2 yaya sce cwonteiece —reversosans ]AMIINUNNUINININIMIINL = $"222 2%\nSpecial !\n$1650)\n2 NICOLA 1B NeteD poe Withpre. — oe*7900089 TT TTT TTTATATATATAVITT TANT] $272 $5459\nPLUCKED EDGES, FREE\nPART\n(YEREL oTrHor gevenemwyzuaverco —overt200  HIIMINIMNNNINUMIUIN S19 $1929\nINVISIBLE CENTER PART\nSubtotal $556.50\nNot rated @ 0% $0.00\nTotal $556.50\nPaid to date $0.00\nTotal Qty: 44\nNo. of Boxes: 2\nREMARKS\nTHANK YOU FOR YOUR ORDER!\nCHECK OUT OUR NEW PRODUCTS AT WWW.VIVICAFOXHAIR.COM | WWW.AMOREMIOHAIR.COM |\nWWW.THEMANNHAIR.COM\n** BY STATE LAW, AMEKOR IS REQUIRED TO PRESERVE YOUR SALES TAX CERTIFICATE.\n** PLEASE SEND US YOUR RESALE CERTIFICATE COPY VIA MAIL, E-MAIL OR FAX.\n

In [40]:
data_1 = pytesseract.image_to_data(image, config=custom_config, output_type=pytesseract.Output.DATAFRAME)

In [30]:
data_1

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1700,2200,-1.000000,NaN
1,2,1,1,0,0,0,79,91,1537,1282,-1.000000,NaN
2,3,1,1,1,0,0,79,91,1537,1282,-1.000000,NaN
3,4,1,1,1,1,0,104,91,111,27,-1.000000,NaN
4,5,1,1,1,1,1,104,91,111,27,96.224319,CLOSE
...,...,...,...,...,...,...,...,...,...,...,...,...
136,5,1,1,1,22,5,1515,1304,101,20,95.551918,315.74
137,4,1,1,1,23,0,80,1350,227,23,-1.000000,NaN
138,5,1,1,1,23,1,80,1350,139,19,87.996628,Box/LBS=>
139,5,1,1,1,23,2,228,1350,65,19,83.619583,#1/25
